In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import tensorflow_hub as hub
import tensorflow_text

import time

In [2]:
SEED = 42

In [3]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

## Funções

In [4]:
def salvar_df_resultado(report, modelo, nome_dataset, tempo_exec):
    
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = modelo
    df_resultado['dataset'] = nome_dataset
    df_resultado['tempo'] = tempo_exec

    df_resultado.to_csv(f'Dados/Resultados/{modelo}/{nome_dataset}_resultado.csv', index = True)
    
    return df_resultado


def salvar_modelo(modelo, dataset, metodo):
    
    nome_arquivo = f"Dados/Modelos/modelo_{dataset}_{metodo}"
    pickle.dump(modelo, open(nome_arquivo, 'wb'))


def carregar_modelo(nome, metodo):
    
    modelo = pickle.load(open(f"Dados/Modelos/modelo_{nome}_{metodo}", 'rb'))
    return modelo


def salvar_modelo_bert(modelo, dataset, metodo):
    
    nome_arquivo = f"Dados/Modelos/{metodo}/{dataset}/"
    modelo.save_pretrained(nome_arquivo)


def carregar_modelo_bert(nome, dataset, metodo):
    
    from transformers import TFAutoModel
    
    modelo = TFAutoModel.from_pretrained(f"Dados/Modelos/{metodo}/{dataset}/")
    return modelo

In [5]:
def pipeline_usem(titulo_1, titulo_2):
    
    # model initialization
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    
    # Calculando embeddings
    embedding_1 = embed(titulo_1)
    embedding_2 = embed(titulo_2)

    similaridade = []
    for i in range(len(embedding_1)):
        # Calculando a matriz de similaridade. Quanto maior o score maior a similaridade.
        similaridade.append(np.inner(embedding_1[i], embedding_2[i]))

    return similaridade

## Universal Sentence Encoder Multilingual

In [6]:
flag_usem = True
if flag_usem == True:

    tam = 10
    for nome, df_treino, df_teste in zip(arquivos, lista_df_treino, lista_df_teste):
        
        X_teste_1 = df_teste["titulo_1"]
        X_teste_2 = df_teste["titulo_2"]
        y_teste = df_teste["match"].to_list()

        inicio_tempo = time.time()
        #similaridade = pipeline_usem(X_teste_1[:tam], X_teste_2[:tam])
        similaridade = pipeline_usem(X_teste_1, X_teste_2)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        y_pred = [1 if num >= 0.5 else 0 for num in similaridade]

        #report = classification_report(y_teste[:tam], y_pred, output_dict = True)
        report = classification_report(y_teste, y_pred, output_dict = True)
        df_resultado = pd.DataFrame(report).transpose()
        df_resultado['modelo'] = nome
        df_resultado['tempo'] = tempo

        df_resultado.to_csv(f'Dados/Resultados/USEM/{nome}_resultado.csv', index = True)